# Start with data importing 
The first file is financials.csv which came from ....

In [292]:
import os
import pandas as pd

df= pd.read_csv('..\\Data\\financials.csv')

print(df.head())

  Symbol                 Name                  Sector   Price  Price/Earnings  \
0    MMM           3M Company             Industrials  222.89           24.31   
1    AOS      A.O. Smith Corp             Industrials   60.24           27.76   
2    ABT  Abbott Laboratories             Health Care   56.27           22.51   
3   ABBV          AbbVie Inc.             Health Care  108.48           19.41   
4    ACN        Accenture plc  Information Technology  150.51           25.47   

   Dividend Yield  Earnings/Share  52 Week Low  52 Week High    Market Cap  \
0        2.332862            7.92       259.77       175.490  1.387211e+11   
1        1.147959            1.70        68.39        48.925  1.078342e+10   
2        1.908982            0.26        64.60        42.280  1.021210e+11   
3        2.499560            3.29       125.86        60.050  1.813863e+11   
4        1.714470            5.44       162.60       114.820  9.876586e+10   

         EBITDA  Price/Sales  Price/Book  \


## 538 Dataset ..
Not really useful yet.

In [293]:
df2 = pd.read_csv('..\\Data\\superbowl_commercials.csv')
print(df2.Brand.unique())
## Only 10 brands and some are the same corp not a very helpful dataset will drop

['E-Trade' 'Budweiser' 'Bud Light' 'Pepsi' 'Doritos' 'NFL' 'Toyota'
 'Coca-Cola' 'Hyundai' 'Kia']


## reference file 
still needs cleaning 
commment has source

In [294]:
df3 = pd.read_csv('..\\Data\\superbowls.txt', sep='\t', lineterminator='\r')
# Text file scraped from https://www.pro-football-reference.com/super-bowl/
print(df3.head())
df3.info()
## prime use here is for the superbowl dates and locations

             Date         SB                Winner Pts                Loser  \
0  \nFeb 12, 2023  LVII (57)    Kansas City Chiefs  38  Philadelphia Eagles   
1  \nFeb 13, 2022   LVI (56)      Los Angeles Rams  23   Cincinnati Bengals   
2   \nFeb 7, 2021    LV (55)  Tampa Bay Buccaneers  31   Kansas City Chiefs   
3   \nFeb 2, 2020   LIV (54)    Kansas City Chiefs  31  San Francisco 49ers   
4   \nFeb 3, 2019  LIII (53)  New England Patriots  13     Los Angeles Rams   

  Pts.1              MVP                Stadium           City       State  
0    35  Patrick Mahomes     State Farm Stadium       Glendale     Arizona  
1    20      Cooper Kupp           SoFi Stadium      Inglewood  California  
2     9        Tom Brady  Raymond James Stadium          Tampa     Florida  
3    20  Patrick Mahomes      Hard Rock Stadium  Miami Gardens     Florida  
4     3   Julian Edelman  Mercedes-Benz Stadium        Atlanta     Georgia  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0

## Main set from kaggle.
This needed some cleaning just to load due to poor data entry in the csv

In [295]:
df4 = pd.read_csv('..\\Data\\superbowl-ads.csv')
# This was cleaned up from Kaggle https://www.kaggle.com/datasets/prondeau/superbowlads

df4.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563 entries, 0 to 562
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Year           563 non-null    object
 1   Product Type   563 non-null    object
 2   Product/Title  562 non-null    object
 3   Plot/Notes     547 non-null    object
dtypes: object(4)
memory usage: 17.7+ KB


As seen the file has a listing of 563 superbowl commercials with the year, product type, product/commmercial title, and sometimes a plot.

We are mostly interested in the product type and getting the parent company from the product.

Look are the year first.

In [296]:
df4.Year.unique()
## Something wrong here with year 

array(['1969', '1970', '1975', '1976', '1979', '1981', '1982', '1984',
       '1988', '1989', '1990', '1992', '1993', '1994', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', 'Food, TV series'],
      dtype=object)

Clearly an issue here. Lets see the specific problem.

In [297]:
df4[df4.Year == 'Food, TV series']
## looks like year should be 2020 and shift the two others over for this entry
## lets make better column names first

,Year,Product Type,Product/Title,Plot/Notes
561,"Food, TV series","Pringles, Rick and Morty",NaN,After Morty enthusiastically enters the living...


The year can be determined to be 2020 and the other columns shifted into place.

Next, let's get better names for the columns and fix the row in question.

In [298]:
df4 = df4.rename(columns={"Product Type":'Product_type', 'Product/Title':'Brand_title','Plot/Notes':'Plot'})

In [299]:
df4.iloc[561] = df4.iloc[561].shift(1)
df4.iloc[561].Year = 2020
df4.iloc[561]

Year                                2020
Product_type             Food, TV series
Brand_title     Pringles, Rick and Morty
Plot                                 NaN
Name: 561, dtype: object

Looks better.

Now, split the `Brand_title` into two columns  one for `Brand` and one for the commercial `title`.

In [300]:
df4[['Brand','Title']] = df4['Brand_title'].str.split('"',expand=True)[[0,1]]
df4.head()

,Year,Product_type,Brand_title,Plot,Brand,Title
0,1969,Car,"Chrysler ""Scuba Diver""[4]",A scuba diver in a dreamy sequence with groovy...,Chrysler,Scuba Diver
1,1970,Car,"Pontiac""Humbler""",A young man pulled up in his GTO to a drive-in...,Pontiac,Humbler
2,1975,Security,"Master Lock""Rifle""",A man attempts to break a Master Lock with a r...,Master Lock,Rifle
3,1976,Copy,"Xerox""Monks""",A monk must make 500 handwritten scriptures fo...,Xerox,Monks
4,1979,Soft drink,"Coca-Cola""Hey Kid, Catch!""","A young kid tries to talk to ""Mean"" Joe Greene...",Coca-Cola,"Hey Kid, Catch!"


Look at the different `Product_type`

In [301]:
product_type_orig = df4.Product_type.unique()
df4.Product_type.unique()

array(['Car', 'Security', 'Copy', 'Soft drink', 'Camera', 'Computer',
       'Restaurant', 'Beer', 'Food', 'Credit card', 'Soft Drink',
       'Footwear', 'Clothing', 'Film', 'Mail', 'Shoes', 'Sports', 'Tire',
       'Website', 'Candy', 'TV', 'Gaming', 'Potato chips', 'PSA',
       'Technology', 'Manufacturing', 'Mobile phone', 'TV show',
       'Video Game', 'Store', 'Truck', 'Product type', 'Car accessories',
       'Adhesives', 'Antifungal medication', 'Feminine care products',
       'Insurance', 'Lingerie', 'Phone', 'Phone accessory', 'Service',
       'Skincare products', 'Software', 'Game', 'Toothpaste',
       'Building materials', 'Cleaner', 'Tax preparation', 'TV series',
       'Video games', 'Web hosting', 'Wireless', 'Airlines', 'Car / Film',
       'Drink', 'Food / Drink', 'Headphones', 'Investments',
       'Laundry detergent', 'Loans', 'Retail', 'TV special', 'Travel',
       'Video game', 'Alcohol', 'Alcohol/TV series', 'Beverage',
       'Smart speaker', 'NFL', 'Video

Let's combine some types. (but save the original names as `product_type_orig`)


In [302]:
df4['Product_type'] = df4['Product_type'].str.replace("Truck","Car").str.replace("Soft Drink", "Soft drink").str.replace("TV special", "TV").str.replace("TV series", "TV").str.replace("TV show", "TV")
df4['Product_type'] = df4['Product_type'].str.replace("Potato chips","Food").str.replace("Drink","Alcohol").str.replace("Beer","Alcohol").str.replace("Candy", "Food")
df4['Product_type'] = df4['Product_type'].str.replace("Copy","Technology").str.replace("Camera","Technology").str.replace("Computer","Technology")
df4['Product_type'] = df4['Product_type'].str.replace("Footwear","Shoes").str.replace("Lingerie","Clothing").str.replace("Food, TV series", "Food")
df4['Product_type'] = df4['Product_type'].str.replace("Phone","Wireless").str.replace("Mobile phone","Wireless").str.replace("TV special", "TV")
df4['Product_type'] = df4['Product_type'].str.replace("Restaurant","Food").str.replace("Shoes","Clothing").str.replace("Mail", "Service")
df4 = df4.drop(df4[df4.Product_type == 'Product type'].index)
df4['Product_type'] = df4['Product_type'].str.replace("Security","Goods").str.replace("Tire", "Goods").str.replace("Laundry detergent","Goods").str.replace("Cleaner","Goods")
df4['Product_type'] = df4['Product_type'].str.replace("Building materials","Goods").str.replace("Antifungal medication", "Goods").str.replace("Skincare products","Goods").str.replace("Cleaner","Goods")
df4['Product_type'] = df4['Product_type'].str.replace("Toothpaste","Goods").str.replace("Adhesives", "Goods").str.replace("Feminine care products","Goods").str.replace("Cleaner","Goods")
df4['Product_type'] = df4['Product_type'].str.replace("Credit card","Service").str.replace("Insurance","Service").str.replace("Loans","Service").str.replace("Tax preparation","Service")
df4['Product_type'] = df4['Product_type'].str.replace("Airlines","Service").str.replace("Investments","Service").str.replace("Travel","Service").str.replace("Web hosting","Service")
df4['Product_type'] = df4['Product_type'].str.replace("Video gaming","Goods").str.replace("Video games","Goods").str.replace("Video game","Goods").str.replace("Video Game","Goods")
df4['Product_type'] = df4['Product_type'].str.replace("Food / Alcohol","Food").str.replace("Food, TV","Food").str.replace("Alcohol/TV","Alcohol")
df4['Product_type'] = df4['Product_type'].str.replace("NFL","Sports").str.replace("Software","Service").str.replace("Smart speaker","Service").str.replace("Retail","Service")
df4['Product_type'] = df4['Product_type'].str.replace("Headphones","Goods").str.replace("Wireless accessory","Goods").str.replace("Game","Goods").str.replace("Beverage","Soft drink")
df4['Product_type'] = df4['Product_type'].str.replace("Manufacturing","Goods").str.replace("Car accessories","Goods").str.replace("Store","Goods").str.replace("Beverage","Soft drink")
df4 = df4.drop(df4[df4.Product_type == 'PSA'].index)
df4[df4.Product_type == 'Services']


,Year,Product_type,Brand_title,Plot,Brand,Title


In [303]:
df4.groupby('Product_type').count().sort_values('Brand', ascending=False)

,Year,Brand_title,Plot,Brand,Title
Product_type,,,,,
Film,140,140,135,140,3
Car,81,81,80,81,80
Alcohol,60,60,58,60,60
Food,58,58,54,58,55
Website,45,45,44,45,44
Goods,33,33,32,33,31
Service,32,32,32,32,30
Soft drink,30,30,29,30,30
TV,23,23,23,23,3


In [341]:
product_type_final = df4.Product_type.unique()
product_type_final 

array(['Car', 'Goods', 'Technology', 'Soft drink', 'Food', 'Alcohol',
       'Service', 'Clothing', 'Film', 'Sports', 'Website', 'TV', 'Gaming',
       'Wireless', 'Car / Film'], dtype=object)

## films is the biggest so let's see what we can do with those.

In [304]:
films = df4[df4.Product_type == 'Film']
films.head()


,Year,Product_type,Brand_title,Plot,Brand,Title
27,1996,Film,Independence Day,TV promo.,Independence Day,None
30,1997,Film,Dante's Peak,TV promo.,Dante's Peak,None
31,1997,Film,The Lost World: Jurassic Park,TV promo.,The Lost World: Jurassic Park,None
32,1997,Film,The Saint,TV promo.,The Saint,None
43,1998,Film,Armageddon,TV promo.,Armageddon,None


First, I used ChatGPT to get the studio that made each movie. The result was copied to Sublime Text keeping the original index as `row` when loaded into the df `filmstudios`.

In [305]:
filmstudios = pd.read_csv('..\\Data\\films.txt', sep='-', lineterminator='\r', header=None)
filmstudios = filmstudios.rename(columns={0:'row',1:'Studio'})
filmstudios.head()

,row,Studio
0,27,20th Century Fox
1,30,Universal Pictures
2,31,Universal Pictures
3,32,Paramount Pictures
4,43,Touchstone Pictures


Merge with `films`

In [306]:
films.tail()

,Year,Product_type,Brand_title,Plot,Brand,Title
528,2019,Film,Us N/A,Promo,Us N/A,None
529,2019,Film,Wonder Park N/A,Trailer,Wonder Park N/A,None
555,2020,Film,F9 Trailer,NaN,F9 Trailer,None
556,2020,Film,The Invisible Man Trailer,NaN,The Invisible Man Trailer,None
557,2020,Film,Mulan Trailer,NaN,Mulan Trailer,None


In [307]:
film_ticker = pd.read_csv('..\\Data\\filmticker.txt', sep='-', lineterminator='\r', header=None)
film_ticker =film_ticker.rename(columns={0:'row',1:'Ticker'})

In [308]:
film_ticker = film_ticker.drop(film_ticker.index[[140,141]])
film_ticker.tail()

,row,Ticker
135,\n528 2019 Monkeypaw Productions,CMCSA
136,\n529 2019 Nickelodeon Movies,VIAC
137,\n555 2020 Universal Pictures,CMCSA
138,\n556 2020 Universal Pictures,CMCSA
139,\n557 2020 Walt Disney Pictures,DIS


In [309]:
film_ticker['row'] = film_ticker['row'].str.split(' ',expand=True)[0]
film_ticker['row'] = film_ticker['row'].str.replace("\n", "")
film_ticker['row'][[103,104]] = [334, 335] 
film_ticker['row'][[103,104]]
film_ticker['row'] = pd.to_numeric(film_ticker['row'], downcast='integer')

Finished cleaning up now merge with films to add the ticker symbol we could probably add in the studio but not much need for that.

In [310]:
films = films.merge(film_ticker, how='left', left_index=True, right_on='row')
films.head()

,Year,Product_type,Brand_title,Plot,Brand,Title,row,Ticker
0,1996,Film,Independence Day,TV promo.,Independence Day,None,27,FOX
1,1997,Film,Dante's Peak,TV promo.,Dante's Peak,None,30,CMCSA
2,1997,Film,The Lost World: Jurassic Park,TV promo.,The Lost World: Jurassic Park,None,31,CMCSA
3,1997,Film,The Saint,TV promo.,The Saint,None,32,VIAC
4,1998,Film,Armageddon,TV promo.,Armageddon,None,43,DIS


In [311]:
films = films.set_index("row")
#films.head()

In [312]:
tvshows = df4[df4.Product_type == 'TV']
#tvshows[['Year','Brand']]

## Use this on ChatGPT to get the tickers again

In [313]:
tv_ticker = pd.read_csv('..\\Data\\tv_ticker.txt', sep=' ', lineterminator='\r', header=None)
tv_ticker = tv_ticker.rename(columns={0:'row',3:'Ticker', 1: 'Year', 2: 'Network'})
tv_ticker = tv_ticker.drop(columns=['Year', 'Network'])
tv_ticker['row'] = pd.to_numeric(tv_ticker['row'], downcast='integer')
tvshows = tvshows.merge(tv_ticker, how='left', left_index=True, right_on='row')
tvshows = tvshows.set_index("row")
#tvshows.head()

## Looks good  move on to the next group

In [314]:
cars = df4[df4.Product_type == 'Car']
#cars[['Year','Brand']][41:]

Use ChatGPT again to generate txt file `car_ticker`

In [315]:
car_ticker = pd.read_csv('..\\Data\\car_ticker.txt', sep=' ', lineterminator='\r', header=None)
car_ticker = car_ticker.rename(columns={0:'row',3:'Ticker', 1: 'Year', 2: 'Car'})
car_ticker = car_ticker.drop(columns=['Year', 'Car'])
car_ticker['row'] = pd.to_numeric(car_ticker['row'], downcast='integer')
cars = cars.merge(car_ticker, how='left', left_index=True, right_on='row')
cars = cars.set_index("row")
#cars.head()

## Now `Alcohol`

In [316]:
alcohol = df4[df4.Product_type == 'Alcohol']
#alcohol[['Year','Brand']]

In [317]:
alcohol_ticker = pd.read_csv('..\\Data\\alcohol_ticker.txt', sep=' ', lineterminator='\r', header=None)
alcohol_ticker = alcohol_ticker.rename(columns={0:'row',3:'Ticker', 1: 'Year', 2: 'beer'})
alcohol_ticker = alcohol_ticker.drop(columns=['Year', 'beer'])
alcohol_ticker['row'] = pd.to_numeric(alcohol_ticker['row'], downcast='integer')
alcohol = alcohol.merge(alcohol_ticker, how='left', left_index=True, right_on='row')
alcohol = alcohol.set_index("row")
#alcohol.head()

## Next `Food`

In [318]:
food = df4[df4.Product_type == 'Food']
#food[['Year','Brand']]

In [319]:
food_ticker = pd.read_csv('..\\Data\\food_ticker.txt', sep='-', lineterminator='\r', header=None)
food_ticker[0] = food_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
food_ticker = food_ticker.rename(columns={0:'row',1:'Ticker'})
food_ticker['row'] = pd.to_numeric(food_ticker['row'])
food = food.merge(food_ticker, how='left', left_index=True, right_on='row')
food= food.set_index("row")
#food.head()

## Next `Website`

In [320]:
website = df4[df4.Product_type == 'Website']
#website[['Year','Brand']]

In [321]:
website_ticker = pd.read_csv('..\\Data\\website_ticker.txt', sep='-', lineterminator='\r', header=None)
website_ticker[0] = website_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
website_ticker = website_ticker.rename(columns={0:'row',1:'Ticker'})
website_ticker['row'] = pd.to_numeric(website_ticker['row'], downcast='integer')
website = website.merge(website_ticker, how='left', left_index=True, right_on='row')
website = website.set_index("row")
#website.head()

## Next `Goods`

In [322]:
goods = df4[df4.Product_type == 'Goods']
#goods[['Year','Brand']]

In [323]:
goods_ticker = pd.read_csv('..\\Data\\goods_ticker.txt', sep='(', lineterminator='\r', header=None)
goods_ticker[0] = goods_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
goods_ticker[1]  = goods_ticker[1].str.replace(')', '')
goods_ticker = goods_ticker.rename(columns={0:'row',1:'Ticker'})
goods_ticker['row'] = pd.to_numeric(goods_ticker['row'], downcast='integer')
goods = goods.merge(goods_ticker, how='left', left_index=True, right_on='row')
goods = goods.set_index("row")
goods.head()

C:\Users\micha\AppData\Local\Temp\ipykernel_12264\444714591.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  goods_ticker[1]  = goods_ticker[1].str.replace(')', '')


,Year,Product_type,Brand_title,Plot,Brand,Title,Ticker
row,,,,,,,
2,1975,Goods,"Master Lock""Rifle""",A man attempts to break a Master Lock with a r...,Master Lock,Rifle,NYSE:FBHS
11,1984,Goods,"Master Lock""The Doubters",NaN,Master Lock,The Doubters,NYSE:FBHS
58,1998,Goods,"Michelin""Drill""",A man goes up to his car and drills a hole in ...,Michelin,Drill,OTC: MGDDF
218,2009,Goods,"Bridgestone""The Potato Heads""",Mr. Potato Head is being nagged by his wife wh...,Bridgestone,The Potato Heads,OTC: BRDCY
246,2010,Goods,"Bridgestone""Killer Whale""",Three men have a killer whale in their small v...,Bridgestone,Killer Whale,OTC: BRDCY


This time ChatGPT inserted the exchange but the colon should make it easy to remove if needed.

## Now `Service`

In [324]:
services = df4[df4.Product_type == 'Service']
#services[['Year','Brand']]

In [325]:
services_ticker = pd.read_csv('..\\Data\\services_ticker.txt', sep='-', lineterminator='\r', header=None)
services_ticker[0] = services_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
services_ticker = services_ticker.rename(columns={0:'row', 1:'Ticker'})
services_ticker['row'] = pd.to_numeric(services_ticker['row'])
services = services.merge(services_ticker, how='left', left_index=True, right_on='row')
services = services.set_index("row")
services.head()



,Year,Product_type,Brand_title,Plot,Brand,Title,Ticker
row,,,,,,,
15,1990,Service,"American Express""The Race""",Paul Newman races a Top-Fuel Dragster on a Min...,American Express,The Race,AXP
42,1998,Service,"Visa""Elephant""",A little girl talks about how she would want a...,Visa,Elephant,V
52,1998,Service,"FedEx""We Apologize""","A colored ""Please Stand By"" test bar is on the...",FedEx,We Apologize,FDX
64,1999,Service,"MasterCard""Toons""",Characters from the Hanna-Barbera cartoons are...,MasterCard,Toons,MA
77,1999,Service,"FedEx""Stanley Cup""",An agent is supposed to deliver the Stanley Cu...,FedEx,Stanley Cup,FDX


## Next `Soft drink`

In [326]:
Soft_drink = df4[df4.Product_type == 'Soft drink']
#Soft_drink[['Year','Brand']]

In [327]:
sodas_ticker = pd.read_csv('..\\Data\\sodas_ticker.txt', sep='-', lineterminator='\r', header=None)
sodas_ticker[0] = sodas_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
sodas_ticker = sodas_ticker.rename(columns={0:'row',1:'Ticker'})
sodas_ticker['row'] = pd.to_numeric(sodas_ticker['row'])
Soft_drink = Soft_drink.merge(sodas_ticker, how='left', left_index=True, right_on='row')
Soft_drink = Soft_drink.set_index("row")
Soft_drink.head()

,Year,Product_type,Brand_title,Plot,Brand,Title,Ticker
row,,,,,,,
4,1979,Soft drink,"Coca-Cola""Hey Kid, Catch!""","A young kid tries to talk to ""Mean"" Joe Greene...",Coca-Cola,"Hey Kid, Catch!",KO
16,1992,Soft drink,"Pepsi""Cindy Crawford""",Two Boys see Cindy Crawford pulling up at a ve...,Pepsi,Cindy Crawford,PEP
36,1997,Soft drink,"Pepsi""Darth Vader""","While watching Star Wars, a little kid is suck...",Pepsi,Darth Vader,PEP
55,1998,Soft drink,"Pepsi""Mosquito""",A CGI fly sucks on a drop of Pepsi. He then gr...,Pepsi,Mosquito,PEP
56,1998,Soft drink,"Pepsi""Goose""","A man does aerial tricks with a goose, and the...",Pepsi,Goose,PEP


## Next `Wireless`

In [328]:
wireless = df4[df4.Product_type == 'Wireless']
#wireless[['Year','Brand']]

In [329]:
wireless_ticker = pd.read_csv('..\\Data\\wireless_ticker.txt', sep='-', lineterminator='\r', header=None)
wireless_ticker[0] = wireless_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
wireless_ticker = wireless_ticker.rename(columns={0:'row',1:'Ticker'})
wireless_ticker['row'] = pd.to_numeric(wireless_ticker['row'])
wireless = wireless.merge(wireless_ticker, how='left', left_index=True, right_on='row')
wireless = wireless.set_index("row")
wireless.head()



,Year,Product_type,Brand_title,Plot,Brand,Title,Ticker
row,,,,,,,
247,2010,Wireless,"Boost Mobile""Shuffle""",Celebrating the 25th anniversary of The Super ...,Boost Mobile,Shuffle,S
350,2015,Wireless,"Sprint """"""Super Apology""""""",A caption-styled ad with curse words changed w...,Sprint,,SFTBY
351,2015,Wireless,"T-Mobile """"""#KimsDataStash""""""",Kim Kardashian asks viewers to help save the d...,T-Mobile,,DTEGY
352,2015,Wireless,"T-Mobile """"""One Upped""""""",Chelsea Handler and Sarah Silverman try to one...,T-Mobile,,DTEGY
386,2016,Wireless,"T-Mobile """"""Drop The Balls""""""",In reference to his incorrect announcement of ...,T-Mobile,,DTEGY


## Next `Technology`

In [330]:
technology = df4[df4.Product_type == 'Technology']
#technology[['Year','Brand']]

In [331]:
tech_ticker = pd.read_csv('..\\Data\\tech_ticker.txt', sep='-', lineterminator='\r', header=None)
tech_ticker[0] = tech_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
tech_ticker = tech_ticker.rename(columns={0:'row',1:'Ticker'})
tech_ticker['row'] = pd.to_numeric(tech_ticker['row'])
technology = technology.merge(tech_ticker, how='left', left_index=True, right_on='row')
technology = technology.set_index("row")
technology.head()



,Year,Product_type,Brand_title,Plot,Brand,Title,Ticker
row,,,,,,,
3,1976,Technology,"Xerox""Monks""",A monk must make 500 handwritten scriptures fo...,Xerox,Monks,XRX
5,1981,Technology,"Polaroid""Now with a Flash""",A couple walks in a park. The man shows that n...,Polaroid,Now with a Flash,PRD
6,1982,Technology,"RadioShack""Color Computer""",A commercial for the TRS-80 computer.,RadioShack,Color Computer,RSHCQ
8,1984,Technology,"Apple Inc.""1984""",In what some call the greatest Super Bowl comm...,Apple Inc.,1984,AAPL
9,1984,Technology,"Atari""Get Started""",NaN,Atari,Get Started,none


## Now `Clothing`

In [332]:
clothing = df4[df4.Product_type == 'Clothing']
#clothing[['Year','Brand']]

In [333]:
clothing_ticker = pd.read_csv('..\\Data\\clothing_ticker.txt', sep='-', lineterminator='\r', header=None)
clothing_ticker[0] = clothing_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
clothing_ticker = clothing_ticker.rename(columns={0:'row',1:'Ticker'})
clothing_ticker['row'] = pd.to_numeric(clothing_ticker['row'])
clothing = clothing.merge(clothing_ticker, how='left', left_index=True, right_on='row')
clothing = clothing.set_index("row")
clothing.head()



,Year,Product_type,Brand_title,Plot,Brand,Title,Ticker
row,,,,,,,
18,1993,Clothing,"Nike""Hare Jordan""",Marvin the Martian takes all the Air Jordans o...,Nike,Hare Jordan,NKE
19,1993,Clothing,"Lee""Inhale""",A man trying to suck in his stomach to button ...,Lee,Inhale,VFC
20,1993,Clothing,"Lee""The Date""",A young woman in her bedroom upstairs wearing ...,Lee,The Date,VFC
26,1994,Clothing,"No Fear""""""Rodeo""""""",The commercial features nine-time World Champi...,No Fear,,none
54,1998,Clothing,"Hilfiger""Have Some Fun""",A fancy woman agent is representing Michael Ri...,Hilfiger,Have Some Fun,PVH


In [334]:
sports = df4[df4.Product_type == 'Sports']
#sports[['Year','Brand']]

In [335]:
sports_ticker = pd.read_csv('..\\Data\\sports_ticker.txt', sep='-', lineterminator='\r', header=None)
sports_ticker[0] = sports_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
sports_ticker = sports_ticker.rename(columns={0:'row',1:'Ticker'})
sports_ticker['row'] = pd.to_numeric(sports_ticker['row'])
sports = sports.merge(sports_ticker, how='left', left_index=True, right_on='row')
sports = sports.set_index("row")
sports.head()



,Year,Product_type,Brand_title,Plot,Brand,Title,Ticker
row,,,,,,,
57,1998,Sports,"NFL""Anthony""",A kid named Anthony is said to play in the NFL.,NFL,Anthony,none
79,1999,Sports,"WWF""HQ""","Famous wrestlers, including Stone Cold Steve A...",WWF,HQ,none
390,2016,Sports,"NFL """"""Super Bowl Babies Choir""""""","A group of ""Super Bowl Babies"", who were all b...",NFL,,none
475,2018,Sports,"2018 Winter Olympics (NBC) """"""Best of U.S.: Li...",Skier Lindsey Vonn prepares for the 2018 Winte...,2018 Winter Olympics (NBC),,CMCSA
476,2018,Sports,2018 Winter Olympics (NBC) N/A A number of pro...,NaN,2018 Winter Olympics (NBC) N/A A number of pro...,None,CMCSA


## Now  `Gaming`

In [336]:
gaming = df4[df4.Product_type == 'Gaming']
#gaming[['Year','Brand']]

In [337]:
gaming_ticker = pd.read_csv('..\\Data\\gaming_ticker.txt', sep='-', lineterminator='\r', header=None)
gaming_ticker[0] = gaming_ticker[0].str.split(' ',expand=True)[0].str.replace('\n','')
gaming_ticker = gaming_ticker.rename(columns={0:'row',1:'Ticker'})
gaming_ticker['row'] = pd.to_numeric(gaming_ticker['row'])
gaming = gaming.merge(gaming_ticker, how='left', left_index=True, right_on='row')
gaming = gaming.set_index("row")
gaming.head()



,Year,Product_type,Brand_title,Plot,Brand,Title,Ticker
row,,,,,,,
127,2001,Gaming,"PlayStation 2""Future""",A future commercial advertises the PS9.,PlayStation 2,Future,SNE
138,2002,Gaming,Madden NFL 2002,TV promo.,Madden NFL 2002,None,EA
155,2004,Gaming,"PlayStation 2""Going Back""",TV promo for Jet Li: Rise to Honor (Full Versi...,PlayStation 2,Going Back,SNE
343,2015,Gaming,"Clash of Clans """"""Revenge""""""",Liam Neeson plans to avenge his loss in the ga...,Clash of Clans,,none
344,2015,Gaming,"Game of War: Fire Age """"""Who I Am""""""",Kate Upton's character leads her troops agains...,Game of War: Fire Age,,none


In [340]:
lexus_blackpanther = df4[df4.Product_type == 'Car / Film']

## Now merge them back together.

In [343]:
tickers_added = pd.concat([films, tvshows, cars, alcohol, food, website, goods, services, Soft_drink, wireless, clothing, technology, sports,gaming ])
tickers_added.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 556 entries, 27 to 344
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Year          556 non-null    object
 1   Product_type  556 non-null    object
 2   Brand_title   556 non-null    object
 3   Plot          539 non-null    object
 4   Brand         556 non-null    object
 5   Title         385 non-null    object
 6   Ticker        556 non-null    object
dtypes: object(7)
memory usage: 34.8+ KB


# Wow that took way more time than I expected!

## Next dataset



In [338]:
df5 = pd.read_csv('..\\Data\\advertisers.txt', sep='\t', lineterminator='\r')
# Text file scraped from http://superadsdb.com/advertisers
df5.head(20)

,Company\Brand,Ads,Industry,Latest,First,Links
0,\n21st Century Studios,14,Movies,2019,1994,NaN
1,\n7 Up,11,Soft Drinks,2001,1992,NaN
2,\n Diet 7 Up,3,Soft Drinks,1992,--,NaN
3,\n84 Lumber,1,Building Supplies,2017,--,NaN
4,\nAccenture,4,Technology,2001,--,NaN
5,\nAcura,3,Automobiles,2016,2001,NaN
6,\n Acura MDX,1,SUVs,2001,--,NaN
7,\n Acura NSX,2,Sports Cars,2016,2012,NaN
8,\nADT,2,Home Security,2020,2019,NaN
9,\nAdvil,5,Pain Relief,2016,1985,NaN
